# LAB 1 – Creación del primer agente `IntakeAgent` (clasificador de solicitudes)

Este notebook forma parte del workshop sobre **Microsoft Agent Framework** aplicado al escenario de un **Service Desk inteligente interno**.

En este primer laboratorio vamos a:

- Configurar el acceso a un modelo **`gpt-4o` de GitHub Models** usando la API compatible con OpenAI.
- Instalar y verificar **Microsoft Agent Framework (Python)**.
- Crear nuestro primer agente, `IntakeAgent`, basado en `ChatAgent`.
- Hacer que el agente:
  - Entienda solicitudes en lenguaje natural.
  - Devuelva una **estructura JSON** que representará un *ticket* de Service Desk.

> ⚠️ Nota: Este notebook asume que dispones de un **Personal Access Token (PAT) de GitHub** con permisos para usar GitHub Models.


## 1. Escenario funcional del LAB 1

Trabajamos en un **Service Desk interno** que recibe peticiones como:

> “Necesito un portátil nuevo con 16 GB de RAM para teletrabajar 3 días a la semana”  
> “Quiero pedir mis vacaciones de agosto”  
> “No puedo conectarme a la VPN de la empresa”

Nuestro objetivo en este lab NO es todavía crear tickets reales, sino construir un agente que:

1. **Entienda** el texto libre del usuario.
2. **Clasifique** la petición en campos estructurados:

```json
{
  "departamento": "IT | RRHH | Facilities | Otro",
  "categoria": "nuevo_equipo | incidencia | vacaciones | certificado | mantenimiento | otro",
  "prioridad": "alta | media | baja",
  "resumen": "Texto corto que resuma la solicitud",
  "detalle": "Texto con más contexto, si es necesario"
}

Instalación de dependencias

En este lab usaremos:

- `agent-framework` – Microsoft Agent Framework para Python. :contentReference[oaicite:4]{index=4}  
- `python-dotenv` (opcional) – si quieres cargar variables desde un fichero `.env`.



## 2. Configuración del modelo `gpt-4o` de GitHub Models

GitHub Models expone modelos de OpenAI (como `gpt-4o`) a través de un **endpoint compatible con la API de OpenAI**. :contentReference[oaicite:2]{index=2}

La idea básica es:

- Usar el cliente oficial `openai` (internamente también lo usa Microsoft Agent Framework para la integración OpenAI).
- Configurar:
  - `OPENAI_API_KEY` → tu **GitHub PAT** (token personal con scope `models:read`).
  - `OPENAI_BASE_URL` → `https://models.inference.ai.azure.com`
  - `OPENAI_CHAT_MODEL_ID` → `gpt-4o` (o el modelo que quieras usar).

Microsoft Agent Framework, en su integración `OpenAIChatClient`, se apoya en el cliente de OpenAI y respeta estas variables de entorno. :contentReference[oaicite:3]{index=3}


In [7]:
import os

# =====================================================
# ⚠️ IMPORTANTE: RELLENA TU FICHERO DE CONFIGURACIÓN
# =====================================================

from dotenv import load_dotenv

load_dotenv()

GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
GITHUB_ENDPOINT = os.getenv("GITHUB_ENDPOINT")
GITHUB_MODEL = os.getenv("GITHUB_MODEL", "openai/gpt-4o")

# Adaptar a las variables esperadas por OpenAIChatClient (Agent Framework)
os.environ["OPENAI_API_KEY"] = GITHUB_TOKEN
os.environ["OPENAI_BASE_URL"] = GITHUB_ENDPOINT
os.environ["OPENAI_CHAT_MODEL_ID"] = GITHUB_MODEL

## 3. Importar Microsoft Agent Framework y verificar configuración

Vamos a importar:

- `ChatAgent` – clase genérica de agente conversacional.
- `OpenAIChatClient` – cliente de chat para modelos tipo OpenAI, en este caso GitHub Models (`gpt-4o`).

También verificaremos que las variables de entorno estén bien configuradas.


In [2]:
from agent_framework import ChatAgent
from agent_framework.openai import OpenAIChatClient



## 4. Creación de `IntakeAgent`: agente de clasificación de solicitudes

Vamos a crear un agente muy sencillo:

- Usa `OpenAIChatClient` configurado con nuestro modelo `gpt-4o`.
- Tiene como objetivo **entender y clasificar** una solicitud de Service Desk.
- Por ahora, sólo generará una respuesta “humana” para comprobar que todo está bien conectado.

Después ajustaremos el comportamiento para que **devuelva JSON estructurado**.


In [8]:
import asyncio

# Instrucciones de alto nivel para el agente de intake
INTAKE_INSTRUCTIONS = '''
Eres un agente de primer nivel de un Service Desk interno.
Tu trabajo es entender las solicitudes de las personas empleadas
y clasificar a qué área pertenecen: IT, RRHH o Facilities.

Responde siempre en español, de forma clara y profesional.
'''

async def create_intake_agent_basic():
    """
    Crea un ChatAgent sencillo usando OpenAIChatClient con GitHub Models (gpt-4o).
    """
    
    chat_client = OpenAIChatClient( 
        base_url=GITHUB_ENDPOINT,
        api_key=GITHUB_TOKEN,
        model_id=GITHUB_MODEL)
    
    agent = chat_client.create_agent(
        name="IntakeAgent",
        instructions=INTAKE_INSTRUCTIONS,
    )
    return agent

async def demo_intake_agent_basic():
    agent = await create_intake_agent_basic()
    thread = agent.get_new_thread()

    consulta = "Necesito un portátil nuevo con 16GB de RAM para teletrabajar 3 días a la semana."
    print(f"👤 Usuario: {consulta}\n")

    result = await agent.run(consulta, thread=thread)
    print("🤖 IntakeAgent (respuesta libre):\n")
    print(result.text)

await demo_intake_agent_basic()


👤 Usuario: Necesito un portátil nuevo con 16GB de RAM para teletrabajar 3 días a la semana.

🤖 IntakeAgent (respuesta libre):

Esta solicitud pertenece al área de **IT**. Recomiendo que la envíes al equipo responsable de asignación de hardware para que puedan gestionar la entrega de un nuevo portátil que cumpla con las especificaciones requeridas.


## 5. Definir el formato de salida JSON del agente

Queremos que `IntakeAgent` devuelva SIEMPRE un JSON con la estructura siguiente:

```json
{
  "departamento": "IT | RRHH | Facilities | Otro",
  "categoria": "nuevo_equipo | incidencia | vacaciones | certificado | mantenimiento | otro",
  "prioridad": "alta | media | baja",
  "resumen": "Texto corto que resuma la solicitud",
  "detalle": "Texto con más contexto, si es necesario"
}


In [4]:

INTAKE_JSON_INSTRUCTIONS = '''
Eres un agente de primer nivel de un Service Desk interno.

Tu tarea es LEER la solicitud del usuario y devolver SIEMPRE un único objeto JSON con esta estructura:

{
  "departamento": "IT | RRHH | Facilities | Otro",
  "categoria": "nuevo_equipo | incidencia | vacaciones | certificado | mantenimiento | otro",
  "prioridad": "alta | media | baja",
  "resumen": "Texto corto que resuma la solicitud",
  "detalle": "Texto con más contexto, si es necesario"
}

Reglas IMPORTANTES:
- No expliques nada fuera del JSON.
- No añadas comentarios ni texto antes o después del JSON.
- Rellena los campos en base a la intención del usuario.
- Si tienes dudas, elige el valor más razonable y utiliza "otro" en categoria/departamento cuando no encaje.
- La prioridad será:
  - "alta" si hay urgencia, bloqueo, caída de servicio o impacto fuerte.
  - "media" en la mayoría de solicitudes estándar.
  - "baja" para dudas generales o temas no urgentes.
'''

async def create_intake_agent_json():
    chat_client = OpenAIChatClient( 
        base_url=GITHUB_ENDPOINT,
        api_key=GITHUB_TOKEN,
        model_id=GITHUB_MODEL)
    agent = chat_client.create_agent(
        name="IntakeAgentJSON",
        instructions=INTAKE_JSON_INSTRUCTIONS,
    )
    return agent

async def demo_intake_agent_json():
    agent = await create_intake_agent_json()
    thread = agent.get_new_thread()

    ejemplos = [
        "Necesito un portátil nuevo con 16GB de RAM para teletrabajar 3 días a la semana.",
        "Quiero pedir mis vacaciones del 1 al 15 de agosto.",
        "No funciona el aire acondicionado en la sala de reuniones 3A.",
        "No puedo conectarme a la VPN y tengo una reunión urgente con un cliente."
    ]

    for consulta in ejemplos:
        print("====================================================")
        print(f"👤 Usuario: {consulta}\n")
        result = await agent.run(consulta, thread=thread)
        print("🤖 IntakeAgentJSON (salida JSON):\n")
        print(result.text)
        print()

await demo_intake_agent_json()


👤 Usuario: Necesito un portátil nuevo con 16GB de RAM para teletrabajar 3 días a la semana.

🤖 IntakeAgentJSON (salida JSON):

{
  "departamento": "IT",
  "categoria": "nuevo_equipo",
  "prioridad": "media",
  "resumen": "Solicitud de portátil nuevo con 16GB de RAM",
  "detalle": "El usuario necesita un portátil con 16GB de RAM para poder teletrabajar 3 días a la semana."
}

👤 Usuario: Quiero pedir mis vacaciones del 1 al 15 de agosto.

🤖 IntakeAgentJSON (salida JSON):

{
  "departamento": "RRHH",
  "categoria": "vacaciones",
  "prioridad": "media",
  "resumen": "Solicitud de vacaciones del 1 al 15 de agosto",
  "detalle": "El usuario solicita vacaciones del 1 al 15 de agosto."
}

👤 Usuario: No funciona el aire acondicionado en la sala de reuniones 3A.

🤖 IntakeAgentJSON (salida JSON):

{
  "departamento": "Facilities",
  "categoria": "incidencia",
  "prioridad": "media",
  "resumen": "Aire acondicionado no funciona en la sala de reuniones 3A",
  "detalle": "El usuario reporta que el a